Question 7

*Group Members*
1. 1741031: Foram Vadher
2. 1741034: Krinali Shah
3. 1741074: Arpit Patel
4. 1741079: Vaishwi Patel

In [0]:
import multiprocessing as mp
import pandas as pd
import numpy as np

In [0]:
import random

check_list = []
cnt = 0
#generate stream ratings
def mygenratorfunc():
    for num in range(100):
        i = random.randrange(0, 12, 1)
        j = random.randrange(0, 60, 1)
        r = random.randrange(0, 5, 1)
        if str(i)+str(j) not in check_list:
            check_list.append(str(i)+str(j))
            
            yield i, j, r
            
            
        

worker

In [0]:
def worker(i,j,r,p,q):

  eta=0.1  #learning rate 
  lamda =0.1 #regularization parameter
  #here i=row ,j=col
  # [i,j]=size(B_pq) #what is i,j see ppt of lecture
  #if R_ij,i,j is recieved
  if((p is None) and (q is None)):
    R[i][j]=r
    #save R_ij
  #if p,q is recieved
  elif((r is None) and (i is None) and (j is None)):
    for row in range(p*n//d,(p+1)*n//d):
      for col in range(q*m//d,(q+1)*m//d):
        U[:,row] = U[:,row] - 2*eta*((np.sum(np.multiply(U[:,row].T,V[:,col]))-r)*V[:,j]+(lamda/m)*U[:,row])
        V[:,col] = V[:,col] - 2*eta*((np.sum(np.multiply(U[:,row].T,V[:,col]))-r)*U[:,row]+(lamda/n)*V[:,col])
  else:
    print("Wait")

  return (p,q)

Master

In [0]:
def collect_result(result):
	global results
	results=result

def master(d,m,n,k):
  count = 100;
  pool = mp.Pool(d)
  results = []
  S = []
  flag_lock_unlock = np.zeros([d,d])
  B_pq = np.zeros([d,d])
  for i,j,r in mygenratorfunc():
    if r!=0:                    #check whether user rating is arrive or not
      B_p = (i*d)//n            #Bpq block which Rij belongs 
      B_q = (j*d)//m
      print("In master",i,j,)
      B_pq[B_p][B_q] =  1       # book keeping information
      
      [a,b]=worker(i,j,r,None,None) # forward (i,j,Rij) to worker
 
      #check for unloacked block and non empty block and put that in S
      for k in range(d):
        for k1 in range(d):
          if(flag_lock_unlock[k][k1]==0):
            
            if (B_pq[k][k1]==1):
              S.append([k,k1])

      #check for S empty or not
      if len(S)!=0:
        index = random.randrange(0, len(S), 1)    #choose random block from the S
        # lock all blocks in row p and column q 
        for m1 in range(d):
          for m2 in range(d):
            
            if(S[index][0]==m1 and S[index][1]==m2):
              flag_lock_unlock[S[index][0]][S[index][1]] = flag_lock_unlock[S[index][0]][S[index][1]]
            elif(S[index][0]==m1):
              flag_lock_unlock[m1][m2] =1
            elif(S[index][1]==m2):
              flag_lock_unlock[m1][m2] =1
        
        pool.apply_async(worker, args=(None,None,None,S[index][0],S[index][1]),callback=collect_result)  #send p q to worker
        pool.apply_async(worker, args=(None,0,None,None,S[index][1]),callback=collect_result)  #send p q to worker
        pool.close()
        pool.join() 

      if len(results)!=0 :    #(p,q) recevied form worker
        for row,col in results:
          p = row
          q = col
          break
        B_pq[p][q]=0    #remove book keeping information

        #Unlock all block in row p and all block in column q 
        for m1 in range(d):
          for m2 in range(d):
            
            if(p==m1):
              flag_lock_unlock[m1][m2] =0
            elif(q==m2):
              flag_lock_unlock[m1][m2] =0
                                  

                                  


#**Main Function**

In [0]:
if __name__=='__main__':
  n = 12 # number of user
  m = 60 # number of item
  k = 10 #number of item feature

  R = np.zeros([n, m])
  U = np.random.randint(5, size=(k, n))
  V = np.random.randint(5, size=(k, m))
  d = 2  #number of worker

  #[U_new , V_new]=parts_U_V(d,k,n,m,U,V)
  #B_pq = block_R(d,m,n,R)


  print("Shape of U before blocking",U.shape)
  print("Shape of V before blocking",V.shape)
  print("Shape of R before blocking",R.shape)
  #print("Shape of U After blocking",U_new.shape)
  #print("Shape of V After blocking",V_new.shape)
  #print("Shape of R After blocking",R_new.shape)


  master(d,m,n,k)
